In [1]:
import igl

In [2]:
#https://github.com/libigl/libigl-tutorial-data

In [3]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time

In [52]:
v, f = igl.read_triangle_mesh('data/arm.obj')
VE,EE,_,BE,CE,_ = igl.read_tgf('data/arm.tgf')
W = igl.read_dmat('data/arm-weights.dmat')
# labels = np.load('data/hand.label.npy').astype(int)
# v -= v.min(axis=0)
# v /= v.max()

In [53]:
handle_vertex_positions = v.copy()
# pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [54]:
def widgets_wrapper():
    # segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    # segment_widget.observe(update_seg, 'value')
    # widgets_dict = dict(s=segment_widget)
    widgets_dict = dict()
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [55]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [63]:

## Widget UI

p = mp.plot(v)
# p = mp.plot(v,f)
p.add_edges(VE,EE, shading={"line_color": "green"});
iw.interact(
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

In [66]:
W

array([[5.76139566e-01, 4.23377394e-01, 1.78198762e-04, 3.04841225e-04],
       [5.53889587e-01, 4.45625716e-01, 1.78814023e-04, 3.05883447e-04],
       [6.64442941e-01, 3.35071519e-01, 1.79124279e-04, 3.06415277e-04],
       ...,
       [9.99300114e-01, 5.15333031e-06, 2.56433055e-04, 4.38299297e-04],
       [9.25684466e-01, 7.38590892e-02, 1.68289386e-04, 2.88155800e-04],
       [9.85045854e-01, 1.45570378e-02, 1.46420915e-04, 2.50686818e-04]])

In [68]:
v

array([[-0.692193 ,  0.084945 ,  0.0399108],
       [-0.693609 ,  0.090783 ,  0.0441324],
       [-0.707763 ,  0.0805516,  0.038791 ],
       ...,
       [-0.891664 ,  0.0875205, -0.0301159],
       [-0.814435 ,  0.161851 , -0.152605 ],
       [-0.831258 ,  0.123607 , -0.141994 ]])